In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

In [ ]:
# 1. 数据加载和预处理
def load_images_from_folder(folder, target_size=(224, 224), plot_type='scatter_plots'):
    images = []
    labels = []
    label = 0  # 为每个发射机分配一个标签
    
    # 遍历文件夹并加载图像
    for tx_id in tqdm(os.listdir(folder), desc=f"Loading {plot_type} images"):
        tx_folder = os.path.join(folder, tx_id)
        if os.path.isdir(tx_folder):
            # 每个发射机目录下有scatter_plots和trajectory_plots
            plot_folder = os.path.join(tx_folder, plot_type)
            if os.path.exists(plot_folder):
                for filename in os.listdir(plot_folder):
                    if filename.endswith('.png'):
                        img_path = os.path.join(plot_folder, filename)
                        img = image.load_img(img_path, target_size=target_size)  # 读取图像并调整大小
                        img_array = image.img_to_array(img)  # 将图像转换为数组
                        img_array = preprocess_input(img_array)  # 进行归一化处理

                        images.append(img_array)
                        labels.append(label)

            label += 1  # 增加发射机的标签

    # 将标签转换为独热编码
    labels = to_categorical(labels)
    return np.array(images), labels

# 使用函数加载散点图图像数据
def load_scatter_images(folder, target_size=(224, 224)):
    return load_images_from_folder(folder, target_size, plot_type='scatter_plots')

# 使用函数加载轨迹图图像数据
def load_trajectory_images(folder, target_size=(224, 224)):
    return load_images_from_folder(folder, target_size, plot_type='trajectory_plots')

In [ ]:
# 2. 构建ResNet模型
def build_resnet_model(num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # 冻结预训练的卷积层
    for layer in base_model.layers:
        layer.trainable = False

    # 添加新的分类层
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.Dense(num_classes, activation='softmax')  # 类别数与发射机数量一致
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# 3. 自定义进度条并训练模型
def train_and_evaluate_model(model, X_train, y_train, X_val, y_val, X_test, y_test, batch_size=32, epochs=20):
    # 训练过程中打印进度
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    # 用 tqdm 生成训练进度条
    with tqdm(total=epochs, desc="Training Progress", unit="epoch") as pbar:
        history = model.fit(
            X_train, y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(X_val, y_val),
            callbacks=[early_stopping],
            verbose=0  # 关闭 Keras 自带的日志打印
        )
        
        # 输出每个epoch的训练和验证结果
        for epoch in range(epochs):
            pbar.update(1)
            # 打印当前 epoch 的损失和准确率
            train_loss = history.history['loss'][epoch]
            val_loss = history.history['val_loss'][epoch]
            train_acc = history.history['accuracy'][epoch]
            val_acc = history.history['val_accuracy'][epoch]
            pbar.set_postfix(train_loss=train_loss, val_loss=val_loss, train_acc=train_acc, val_acc=val_acc)

    print("\nTraining completed.")
    print(f"Best training accuracy: {max(history.history['accuracy'])}")
    print(f"Best validation accuracy: {max(history.history['val_accuracy'])}")
    
    # 在测试集上评估模型
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f"Test accuracy: {test_acc}")
    
    return history, test_acc

In [ ]:
# 4. 比较训练效果
def compare_scatter_trajectory_training(folder, batch_size=32, epochs=20):
    # 加载数据
    print("Loading scatter plot images...")
    X_scatter, y_scatter = load_scatter_images(folder)
    print("Loading trajectory plot images...")
    X_trajectory, y_trajectory = load_trajectory_images(folder)
    
    # 划分训练集、验证集和测试集
    X_train_scatter, X_test_scatter, y_train_scatter, y_test_scatter = train_test_split(X_scatter, y_scatter, test_size=0.2, random_state=42)
    X_train_trajectory, X_test_trajectory, y_train_trajectory, y_test_trajectory = train_test_split(X_trajectory, y_trajectory, test_size=0.2, random_state=42)

    X_train_scatter, X_val_scatter, y_train_scatter, y_val_scatter = train_test_split(X_train_scatter, y_train_scatter, test_size=0.2, random_state=42)
    X_train_trajectory, X_val_trajectory, y_train_trajectory, y_val_trajectory = train_test_split(X_train_trajectory, y_train_trajectory, test_size=0.2, random_state=42)

    # 创建模型
    num_classes = len(os.listdir(folder))  # 类别数等于发射机数量
    
    # 训练散点图模型
    scatter_model = build_resnet_model(num_classes)
    print("\nTraining scatter plot model...")
    scatter_history, scatter_test_acc = train_and_evaluate_model(
        scatter_model, X_train_scatter, y_train_scatter, X_val_scatter, y_val_scatter, X_test_scatter, y_test_scatter, batch_size, epochs
    )
    
    # 训练轨迹图模型
    trajectory_model = build_resnet_model(num_classes)
    print("\nTraining trajectory plot model...")
    trajectory_history, trajectory_test_acc = train_and_evaluate_model(
        trajectory_model, X_train_trajectory, y_train_trajectory, X_val_trajectory, y_val_trajectory, X_test_trajectory, y_test_trajectory, batch_size, epochs
    )

    # 输出比较结果
    print("\nComparison of Test Accuracies:")
    print(f"Scatter plot model test accuracy: {scatter_test_acc}")
    print(f"Trajectory plot model test accuracy: {trajectory_test_acc}")

In [ ]:
# 使用示例：比较散点图和轨迹图的训练效果
folder = "../../IQ_signal_plots"  # 图像存储的文件夹
compare_scatter_trajectory_training(folder, batch_size=32, epochs=20)